# Run model inference

- use the pre-trained model (default) or set `MODEL_FILE` to point to another model
- set `TEST_FILE` to the path of a chord .wav file. It should contain only one chord and be at least 3 seconds long
- the included `testMin.wav` file will be used by default. It does not stem from the dataset used for training.

In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import img_to_array
import librosa
import IPython

MODEL_FILE = 'major-minor-model.h5'
TEST_FILE = 'testMin.wav'
SHAPE = (1025, 94)

def load_chord(input_path):
    [y, _] = librosa.load(input_path, sr=16000)
    max_length = 16000 * 3
    y = librosa.util.fix_length(y, size=max_length)
    spectrogram = librosa.stft(y)
    amplitudes_db = librosa.amplitude_to_db(np.abs(spectrogram), ref=np.max)
    normalized = (amplitudes_db - amplitudes_db.min()) / (amplitudes_db.max() - amplitudes_db.min())
    as_image = (normalized * 255).astype(np.uint8)
    image = Image.fromarray(as_image)
    resized_image = image.resize(SHAPE[::-1], Image.LANCZOS)  # Note: SHAPE is (height, width), PIL expects (width, height)
    img_array = img_to_array(resized_image)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.

    return img_array


model = tf.keras.models.load_model(MODEL_FILE)

print(model.summary())
img = load_chord(TEST_FILE)
[audio_file, sr] = librosa.load(TEST_FILE)
pred = model.predict(img, verbose=1)
print(pred)

if pred > 0.5:
    print("Predicted minor chord")
else:
    print("Predicted major chord")

IPython.display.Audio(TEST_FILE)